In [301]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import wxconv
import glob
from wxconv import WXC
import codecs
import os,subprocess

In [73]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

In [180]:
d = {'word':[],'grammar':[],'meaning':[]}
mnf = {}
ct = 0
for i in range(318):
    URL = "https://sanskrit.inria.fr/MW/" + str(i) + ".html"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", {"class": "minayeff"})
    if results:
        hr_s  = results.find_all("hr")
        #hr1 = results.find("hr")
        #hr1 = hr_s[6]
        for hr1 in hr_s:
            word = ''
            try:
                while hr1.find_next().name != 'hr':

                    if hr1.find_next().name == 'p':
                        p = hr1.find_next()

                        if word == '' :
                            #print(['ooo',p])
                            word = p.find("span", {"class": "Deva"}).text
                            #print(['wordooo',word])
                        if p.find("span", {"class": "Deva"}) == None and word =='':
                            ct += 1
                            print('opopo')
                        if p.find("a", {"class": "Red"}) and p.find("a", {"class": "Blue"}) and p.find("span", {"class": "Deva"})==None:
                            word_iast = p.find("a", {"class": "Blue"}).text
                            word = transliterate(word_iast, sanscript.IAST, sanscript.DEVANAGARI)
                        pos_s = p.find_all("a", {"class": "Red"})
                        #print(['ppppp',pos_s])
                        grammar = ''
                        for pos in pos_s:
                            grammar += pos.text
                        meaning_tags = p.find_all("span", {"class": "text"})
                        meaning_s = []
                        for m in meaning_tags:
                            meaning_s.append(m.text)
                        if len(meaning_s) == 0:
                            mhtml = '<b>MNF</b>'
                            p_children = p.findChildren(recursive = False)
                            if p_children:
                                temp = p_children[-1].next_sibling
                                mean = ''
                                while temp:
                                    #print([word,mean])
                                    mean += temp
                                    temp = temp.next_sibling
                                meaning_s = [mean]
                                #print([word,mean])
                            else:
                                meaning_s = [p.text]
                                #print([word,p.text])
                            #meaning_s = [BeautifulSoup(mhtml, "html.parser")]

                        #if len(meaning_s) == 0:
                            #print(p.text)
                        for m in meaning_s:
                            d['word'].append(word)
                            d['grammar'].append(grammar)
                            d['meaning'].append(m)
                    hr1 = hr1.find_next()
                    if hr1 == None:
                        break
            except:
                #print(i)
                continue
    else:
        print(i)
        #if hr1.name == 'p':
            #print(hr1)
    #get_word_meaning(results,d,i)

0


In [215]:
df = pd.DataFrame.from_dict(d)
df

,word,grammar,meaning
0,अकार,m.,the letter or sound
1,अ,,a vocative particle
2,अ,,O Vishṇu
3,अ,,"interjection of pity , Ah!"
4,अ,,before a vowel
...,...,...,...
357666,ह्वायक,m.f.n.,calling
357667,ह्वायकीय,,] Lit. ib.
357668,ह्वाययितव्य,,] .
357669,ह्वायिन्,m.f.n.,"calling , invoking , challenging"


In [216]:
df.meaning = df.meaning.apply(lambda x : x.strip(' '))
df['word'] = df.word.apply(lambda x : x.strip(' '))
def checkVerb(row):
    meaning = row['meaning']
    if meaning[:6] != 'to be ' and meaning[:3] == 'to ':
        return True
    else:
        return False

In [217]:
#df[df.grammar.isdigit()]
df['isverb'] = df.apply(lambda x : checkVerb(x),axis=1)
df['gisdigit'] = df.grammar.apply(lambda x: x.isdigit())
df[df.isverb == True]

,word,grammar,meaning,isverb,gisdigit
45,अंश्,10,"to divide , distribute",True,True
83,अंशीकृ,,to share.,True,False
160,अंह्,110,"to go , set out , commence",True,True
161,अंह्,110,to approach,True,True
162,अंह्,110,to send,True,True
...,...,...,...,...,...
357601,ह्वॄ,1,to go wrong or astray,True,True
357630,ह्वे,1,"to call , call upon , summon , challenge , inv...",True,True
357631,ह्वे,1,"to emulate , vie with",True,True
357635,ह्वे,1,to cause anyone,True,True


In [219]:

def grammar_verb(row):
    isverb = row['isverb']
    grammar = row['grammar']
    isdigit = row['gisdigit']
    if isverb:
        return 'v.'
    elif isdigit:
        return ''
    else:
        return grammar
df['grammar'] = df.apply(lambda x: grammar_verb(x),axis =1)

In [220]:
gdict = {}
def fill_grammar(row):
    word= row['word']
    grammar = row['grammar']
    if word in gdict.keys() and grammar != '' and grammar != gdict[word]:
        gdict[word] = grammar
    if grammar == '' and word in gdict.keys():
        return gdict[word]
    if word not in gdict.keys() and grammar != '':
        gdict[word] = grammar
    return grammar

In [221]:
df['grammar'] = df.apply(lambda x : fill_grammar(x),axis =1)

In [223]:
df.grammar.unique()

array(['m.', '', 'm.f.n.', 'v.', 'n.', 'f.', 'm.n.', 'm.f.', 'f.n.',
       'm.f.n.m.f.n.', 'n.f.', 'f.f.'], dtype=object)

In [228]:
xd = {'word':[],'meaning':[],'grammar':[]}
def expand_d(row):
    try:
        
        word = row['word']
        meaning = row['meaning']
        pos = row['grammar']
        if meaning == None:
            print('oooo')
        meaning_s1 = meaning.split(';')
        meaning_s2 = []
        for i in meaning_s1:
            s2 = i.split(',')
            for j in s2:
                xd['word'].append(word)
                xd['meaning'].append(j)
                xd['grammar'].append(pos)
    except:
        print([word,meaning])

In [229]:
df.apply(lambda row:expand_d(row),axis =1 )

0         None
1         None
2         None
3         None
4         None
          ... 
357666    None
357667    None
357668    None
357669    None
357670    None
Length: 357671, dtype: object

In [376]:
xdf = pd.DataFrame.from_dict(xd)
xdf

,word,meaning,grammar
0,अकार,the letter or sound,m.
1,अ,a vocative particle,
2,अ,O Vishṇu,
3,अ,interjection of pity,
4,अ,Ah!,
...,...,...,...
516516,ह्वाययितव्य,] .,
516517,ह्वायिन्,calling,m.f.n.
516518,ह्वायिन्,invoking,m.f.n.
516519,ह्वायिन्,challenging,m.f.n.


In [377]:
indexNames = xdf[xdf['meaning']==''].index
xdf = xdf.drop(indexNames).reset_index()

In [378]:
def handle_1(row):
    return re.sub("[\(\[].*?[\)\]]", "", row).strip(' ')
def handle_2(row):
    return row.split(')')[-1]
def handle_3(row):
    return row.split(']')[-1]
def handle_4(row):
    return row.split('[')[0]
def handle_5(row):
    return row.split('(')[0]
def handle_9(row):
    meaning = row['meaning']
    word = row['word']
    grammar = row['grammar']
    if 'N.' in meaning or 'n.' in meaning or 'The ' in meaning:
        return 'n.'
    else:
        return grammar
def handle_6(row):
    if 'N. of ' in row:
        return row.replace('N. of ','').strip(' ')
    else:
        return row.strip(' ')
def handle_7(row):
    row = row.replace('\t','')
    row = row.replace('\n','')
    row = row.replace('˚','')
    row = row.strip('"')
    row = row.strip("'")
    row = row.strip('.')
    row = row.strip('*')
    row = row.strip('-')
    row = row.strip('=')
    if len(row)>3:
        if row[:3] == 'or ':
            row =  row[3:]
        if row[:3] == 'An ' or row[:3] == 'an ':
            row =  row[3:]
        if row[:2] == 'A ':
            row = row[2:]
        if row[:2] == 'a ':
            row = row[2:]
        if row[:4] == 'The ' or row[:4] == 'the ':
            row = row[4:]
    
            
    row = row.replace(' a ', '')
    return row
def handle_8(row):
    ret = row.replace('.','')
    if ret.isdigit():
        return ''
    else:
        return row

In [379]:
xdf.meaning = xdf.meaning.apply(lambda x : handle_1(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_2(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_3(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_4(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_5(x))
xdf.pos = xdf.apply(lambda x : handle_9(x),axis=1)
xdf.meaning = xdf.meaning.apply(lambda x : handle_6(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_7(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_8(x))

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_27131/1367374564.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  xdf.pos = xdf.apply(lambda x : handle_9(x),axis=1)


In [380]:
indexNames = xdf[xdf['meaning']==''].index
xdf = xdf.drop(indexNames)

In [381]:
unw = ['. —','a.','adv.','Rv.','Mb.','+','ind.','q.v.','N.','n.','f.',' (=','&c.','&c','etc','= ~','(= ~',')','inv.','A.','U.','P.','Ā.','&','[',']','=',':',"/",'!','$']
def clean_set(m):
    ret = m
    ret = re.sub("[\(\[].*?[\)\]]", "", m)
    for i in unw:
        ret = ret.replace(i,'')
    
    return ret.strip(' ')
    

In [382]:
def rem_lit(row):
    return re.sub("Lit\..*$", "", row)

In [383]:
xdf.meaning = xdf.meaning.apply(lambda x : clean_set(x))
xdf.meaning = xdf.meaning.apply(lambda x : rem_lit(x))

In [384]:
indexNames = xdf[xdf['meaning']==''].index
xdf = xdf.drop(indexNames).reset_index()

In [385]:
xxd = {'word':[],'meaning':[],'grammar':[]}

def or_split(row):
    meaning = row['meaning']
    pos = row['grammar']
    word = row['word']
    split_sp = meaning.split(' ')
    if len(split_sp) == 3 and split_sp[1]=='or':
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[0])
        xxd['grammar'].append(pos)
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[2])
        xxd['grammar'].append(pos)
    elif len(split_sp) ==4 and split_sp[0] == 'To' and split_sp[2] == 'or' and pos == 'verb':
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[1])
        xxd['grammar'].append(pos)
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[3])
        xxd['grammar'].append(pos)
    else:
        xxd['word'].append(word)
        xxd['meaning'].append(meaning.strip(' '))
        xxd['grammar'].append(pos)


In [386]:
xdf.apply(lambda row: or_split(row),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
468419    None
468420    None
468421    None
468422    None
468423    None
Length: 468424, dtype: object

In [428]:
xxdf = pd.DataFrame.from_dict(xxd)

xxdf['grammar'] = xxdf.apply(lambda x: 'v.' if checkVerb(x) else x['grammar'],axis=1)

,word,meaning,grammar
8392,अध्यवसो,to undertake,v.
8395,अध्यवसो,to determine,v.
8417,अध्यवसेय,only to be conceived in the mind,m.f.n.
8418,अध्यवहन्,to thrash upon,v.
8421,अध्यस्,to throw or place over or upon,v.
8422,अध्यस्,to attribute or impute wrongly,v.
8429,अध्यह्,to speak on behalf of,v.
8431,अध्याक्रम्,to attack,v.
8432,अध्याक्रम्,to choose,v.
8434,अध्यागम्,to meet with,v.


In [429]:
def rem_verb_pref(row):
    pos = row['grammar']
    meaning = row['meaning']
    if pos =='v.' and meaning[:3] == 'to ':
        return meaning[3:]
    return meaning

In [432]:
xxdf['meaning'] = xxdf.apply(lambda x:rem_verb_pref(x),axis=1)

In [710]:
sxxdf = xxdf[xxdf.meaning.apply(lambda x : ' ' not in x)]
sxxdf

,word,meaning,grammar
0,अकार,letter,m.
1,अकार,sound,m.
5,अ,Ah,
6,अ,beforevowel,
7,अ,exc,
...,...,...,...
470285,ह्वान,noise,n.
470287,ह्वायक,calling,m.f.n.
470288,ह्वायिन्,calling,m.f.n.
470289,ह्वायिन्,invoking,m.f.n.


In [711]:
def checkdigit(x):
    for i in x:
        if i.isdigit() == True:
            return True
    return False

In [712]:
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x:checkdigit(x)==False)]

In [713]:
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x: len(x)>1)]
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x: x.isdigit() == False)]
sxxdf['meaning'] = sxxdf.meaning.apply(lambda x : x.strip('.'))

In [714]:
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x: '.' not in x)]

In [715]:
def check_length(row):
    if len(row)==3:
        return True
    return False
unw_two = ['Mb', 'pe', 'Mu', 'Ts','IV', 'VI','Goth','Hib','xiii','also','with',
       'as', 'Pt', 'Av', 'bl', 'Si', 'Ki','Intens','Cit','un-','Gov','on','Cambro-Brit'
        'Śi', 'Śr', 'II', 'Mv', 'Ms', 'Dk','\xa0wk','\xa0\xa0\xa0','Mbh','xix','ind',"id'",
       'Uṇ', 'Bk', 'Ku', 'Mk', 'Su', 'ma','for','of','in','but','and','in','ibc','Inf','VRJ',
       'Ks', 'Bh', 'Pr', 'th', 'Bg', 'Bv', 'fh', 'Ti', 'EI','irreg','by','me','?fr','Vāc','TUP',
       'MS', 'V-', 'Vb', 'Ch',  'fu',  'st','w.r','prep','instr','In','ni','is','Lit','accord',
       'fe', 'AV', 'MW', 'Ṛs', 'vh', 'Tv', 'pi', 'Nm','ifc','Aor','Impv','mc','Mr','pf','MBh','RhP',
     'R-', 'YI', 'Cf', 'fa', 'M/', 'BR', 'ff','Opt','see','aor','Comm','Lith','cramp','onomat','iii',
       'bo', 'XX', 'ru', 'se', 'si', 'pl', 'gr', 'Rv','Imper','fhing','Ved','Russ','patr','Pers','xvi',
       'Ṛv', 'ŚB', 'eg', 'du', 'ho','\xa0\xa0','or','Ia', 'cṛ', 'MS', 'lb', 'Ry', 'Ma', 'UP','Nā','to','vii',
       'cr', 'fo', 'en', 'jb', 'Pr', 'mt', 'ld', 'tn','Sl', 'Bn', 'VS', 'GS', 'fi', 'Gr', 'ga', 'ig', 'Vp', 'pp',
       'us', 'AP', 'Av', 'st', 'sv', 'Am', 'Pā', 'Pl', 'cī', 'Kā', 'ft','viii','ix','xiv','id','viz','ib',
       'mc', 'ar', 'ch','ii', 'ax', 'xx', 'if', 'vc', 'ic', 'ff', 'ed', 'lv','perf','The','Voc','Sch','Caus',
       'un', 'pr', 'we', 'Iv', 'ef', 'Vr', 'Up', 'cl', 'Ap', 'yi', 'p.','Sk','Gk', 'K.', 'xv', 'ix', 'an', 'Fr','kh', 'ph', 'am', 'm.', 'sc', 'II', 'TS', 'du', 'mf', 'vi', 'IW',
       'fh','cf', '\xa0\xa0', 'wk', 'iv','Zd','id','sg', 'c.', 'ri', 'xi', 'Mn','fr','ep','exc','Lat','Eng']
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x: x not in unw_two)]

In [716]:
abbr = []
for l in getLines('abbr.txt','\n'):
    l = l.replace('HI>','HI>  ')
    b = BeautifulSoup(l,"html.parser")
    if b.find('hi'):
        abbr.append(re.findall(r'\ (.*?)\=' , str(b.find('hi'))))
abbr2 = []
for i in abbr:
    if len(i) == 1:
        abbr2.append(i[0])
abbr2 = [j.replace('{%or%}',',')for j in abbr2]
abbr2 = [j.replace(' ','')for j in abbr2]
abbr2 = ','.join(abbr2)
abbr2 = abbr2.split(',')
abbr2 = [j.replace('.','') for j in abbr2]
abbr2

['A',
 'abl',
 'ablc',
 'acc',
 'accc',
 'accord',
 'ƒÄdi-p',
 'adj',
 '√Üol',
 'alg',
 'AnglSax',
 'anom',
 'Aor',
 'Arab',
 'arithm',
 'Arm',
 'Armor',
 'Armen',
 'astrol',
 'astron',
 'Atharva-v',
 'BhƒÅgavata-P',
 'Bha·π≠·π≠i-k',
 'Boh',
 'Bohem',
 'BR',
 'Br',
 'Bret',
 'Buddh',
 'c',
 'Cambro-Brit',
 'Caus',
 'cf',
 'chap',
 'cl',
 'Class',
 'col',
 'cols',
 'comm',
 'comp',
 'comps',
 'compar',
 'Cond',
 'Condit',
 'cons',
 'dat',
 'datc',
 'defect',
 'Desid',
 'dimin',
 'Dor',
 'du',
 'ed',
 'edit',
 'eg',
 'Eng',
 'Ep',
 'ep',
 'epith',
 'esp',
 'etym',
 'explet',
 'f',
 'fem',
 'fr',
 'Fut',
 'Ga√´l',
 'gen',
 'genc',
 'gend',
 'geom',
 'Germ',
 'Goth',
 'Gr',
 'Gram',
 'gram',
 'Hib',
 'Hind',
 'Icel',
 'ie',
 'impers',
 'Impf',
 'Impv',
 'ind',
 'Inf',
 'infin',
 'inst',
 'instc',
 'Intens',
 'Ion',
 'Island',
 'KirƒÅt',
 'KirƒÅtƒÅrj',
 'KumƒÅra-s',
 'Lat',
 'lat',
 'Lett',
 'lit',
 'Lith',
 'loc',
 'locc',
 'long',
 'm',
 'masc',
 'MahƒÅ-bh&amp;c',
 'mathem',
 'medic',
 'M

In [717]:
sxxdf = sxxdf[sxxdf.meaning.apply(lambda x: x not in abbr2)]

In [718]:
sxxdf[sxxdf.meaning.apply(lambda x : x[-2:]=='ly')][4200:4220]

,word,meaning,grammar
408025,सत्य,really,n.
408059,सत्य,truly,n.
408061,सत्य,certainly,n.
408062,सत्य,verily,n.
408063,सत्य,necessarily,n.
408130,सत्यतस्,truly,
408131,सत्यतस्,really,
408406,सत्यसंकाश,likely,m.f.n.
408806,सन्मान,frequently,n.
408828,सतत्त्वतस्,really,


In [730]:
sxxdf[sxxdf.grammar=='adv']

,word,meaning,grammar
177,अकस्मात्,accidentally,adv
178,अकस्मात्,suddenly,adv
182,अकाण्ड,causelessly,adv
183,अकाण्ड,unexpectedly,adv
188,अकामतस्,unintentionally,adv
...,...,...,...
190861,हुरस्,secretly,adv
191164,हृष्टवत्,cheerfully,adv
191223,हेतु,rarely,adv
191429,हेला,sportively,adv


In [720]:
adv_df = pd.read_csv('ending-ly-not-adverbs.csv')
not_adv = adv_df.Word.tolist()

In [721]:
def mod_grammar(row):
    grammar = row['grammar']
    meaning = row['meaning']
    if meaning[-2:] == 'ly' and meaning not in not_adv:
        return 'adv'
    elif grammar == 'm.f.n.' or grammar =='m.f.n.m.f.n.' or grammar =='m.f.':
        return 'adj'
    elif grammar == 'v.':
        return 'verb'
    else:
        return 'noun'

In [722]:
sxxdf['grammar'] = sxxdf.apply(lambda x: mod_grammar(x),axis=1)

In [723]:
sxxdf = sxxdf.reset_index()

In [724]:
sxxdf = sxxdf.drop(['index'],axis=1)

In [725]:
sxxdf

,word,meaning,grammar
0,अकार,letter,noun
1,अकार,sound,noun
2,अ,Ah,noun
3,अ,beforevowel,noun
4,अ,endless,noun
...,...,...,...
191780,ह्वान,noise,noun
191781,ह्वायक,calling,adj
191782,ह्वायिन्,calling,adj
191783,ह्वायिन्,invoking,adj


In [726]:
sxxdf.to_csv('mw-sansdict-v2.txt',sep='\t')
sxxdf.to_csv('mw-sansdict-v2.csv',sep=',')


In [528]:
def getLines(file,splitby):
    with codecs.open(file, 'r', encoding="utf-8") as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
                    data = data.strip('\n')
    lines = data.split(splitby)
    return lines

In [578]:
re.find(r'\ (.*?)\='," A. =")

['A. ']

In [731]:
df2 = pd.read_csv('mw_sanskrit-eng_v1_dict.txt',sep='\t')

In [732]:
df2[100:120]

,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
100,अंहति,trouble,noun,_trouble_n_1,LBL: h* ARG0: x*
101,अंहति,illness,noun,_illness_n_1,LBL: h* ARG0: x*
102,अंहति,gift,noun,_gift_n_of-to,LBL: h* ARG0: x* ARG1: i* ARG2: i*
103,अंहती,gift,noun,_gift_n_of-to,LBL: h* ARG0: x* ARG1: i* ARG2: i*
104,अंहस्,anxiety,noun,_anxiety_n_1,LBL: h* ARG0: x*
105,अंहस्,trouble,noun,_trouble_n_1,LBL: h* ARG0: x*
106,अंहस्,sin,noun,_sin_n_1,LBL: h* ARG0: x*
107,अंहिति,gift,noun,_gift_n_of-to,LBL: h* ARG0: x* ARG1: i* ARG2: i*
108,अंहिति,donation,noun,_donation_n_1,LBL: h* ARG0: x*
109,अंहु,narrow,adj,_narrow_a_1,LBL: h* ARG0: e* ARG1: x*


In [733]:
unw_concs = ["compound","number_q","udef_q","pronoun_q","proper_q","person","parg_d","generic_entity","numbered_hour",
 "person","def_implicit_q","loc_nonsp","time_n","superl","def_explicit_q","poss","comp"]
df2  = df2[df2.MRS_Concept.apply(lambda x:x not in unw_concs)]

In [738]:
df2 = df2[df2.Sans_Label.apply(lambda x : 'Oops!' not in x)]

In [739]:
# adding count at the end of the words and meanings
mrs_word_cnt = {}
mrs_mean_cnt = {}
def count_word(row):
    word = row['Sans_Label']
    if word in mrs_word_cnt.keys():
        mrs_word_cnt[word] +=1
        return word+'_'+ str(mrs_word_cnt[word])
    else:
        mrs_word_cnt[word] =1
        return word+'_'+ str(mrs_word_cnt[word])
def count_meaning(row):
    meaning = row['Eng_Label']
    if meaning in mrs_mean_cnt.keys():
        mrs_mean_cnt[meaning] +=1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
    else:
        mrs_mean_cnt[meaning] =1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
        

In [740]:
ndf2 = df2
ndf2['Sans_Label'] = ndf2.apply(lambda row:count_word(row),axis =1)
ndf2['Eng_Label'] = ndf2.apply(lambda row:count_meaning(row),axis =1)

ndf2

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_27131/2183572935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf2['Sans_Label'] = ndf2.apply(lambda row:count_word(row),axis =1)
/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_27131/2183572935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf2['Eng_Label'] = ndf2.apply(lambda row:count_meaning(row),axis =1)


,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
0,word_1,meaning_1,grammar,unknown,LBL: h* ARG0: e* ARG: x*
2,word_2,meaning_2,grammar,_meaning_n_1,LBL: h* ARG0: x*
3,अकार_1,letter_1,noun,_letter_n_of,LBL: h* ARG0: x* ARG1: i*
4,अकार_2,sound_1,noun,_sound_n_of,LBL: h* ARG0: x* ARG1: i*
5,अ_1,Ah_1,noun,named,",CARG:_VALUES"
...,...,...,...,...,...
235727,ह्वायक_1,calling_63,adj,_call_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: p*
235728,ह्वायिन्_1,calling_64,adj,_call_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: p*
235730,ह्वायिन्_2,invoking_19,adj,_invoke_v_1,LBL: h* ARG0: e* ARG1: i* ARG2: p*
235731,ह्वायिन्_3,invoking_20,adj,nominalization,LBL: h* ARG0: x* ARG1: h*


In [742]:
#ndf2 = ndf2[2:]
ndf2.reset_index(inplace = True)

In [745]:
ndf2.drop(['index'],inplace=True,axis=1)

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_27131/414059957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf2.drop(['index'],inplace=True,axis=1)


In [746]:
ndf2

,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
0,अकार_1,letter_1,noun,_letter_n_of,LBL: h* ARG0: x* ARG1: i*
1,अकार_2,sound_1,noun,_sound_n_of,LBL: h* ARG0: x* ARG1: i*
2,अ_1,Ah_1,noun,named,",CARG:_VALUES"
3,अ_2,beforevowel_1,noun,_beforevowel/NN_u_unknown,LBL: h* ARG0: x*
4,अ_3,likeBrahman_1,noun,named,",CARG:_VALUES"
...,...,...,...,...,...
185628,ह्वायक_1,calling_63,adj,_call_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: p*
185629,ह्वायिन्_1,calling_64,adj,_call_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: p*
185630,ह्वायिन्_2,invoking_19,adj,_invoke_v_1,LBL: h* ARG0: e* ARG1: i* ARG2: p*
185631,ह्वायिन्_3,invoking_20,adj,nominalization,LBL: h* ARG0: x* ARG1: h*


In [747]:
ndf2.to_csv('mw_sanskrit-eng-v1.txt',sep='\t')
ndf2.to_excel('mw_sanskrit-eng-v1.xlsx')